## Realizando as importações

In [26]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [27]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [28]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [29]:
import os

os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [30]:
!pip install -q findspark

In [31]:
import findspark

In [32]:
findspark.init()

## Iniciando uma sessão Spark

In [36]:
import pyspark
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()
spark = SparkSession.builder.getOrCreate()

## Criando DataFrame

In [45]:
dataframe1 = spark.createDataFrame(

[ #create your data

("Monitor LCD", 25000),
("TV LED 44", 35000),
("Palio", 21000)
],
["Produto", "Preço"] #create name columns
)

dataframe1.show()
dataframe1.printSchema()

+-----------+-----+
|    Produto|Preço|
+-----------+-----+
|Monitor LCD|25000|
|  TV LED 44|35000|
|      Palio|21000|
+-----------+-----+



In [47]:
from datetime import datetime, date

dataframe2 = spark.createDataFrame([

(563214, "Maria", date(2000, 8, 9)),
(236541, "João", date(1997, 12, 10)),
(547812, "José", date(1990, 2, 21))

], ["CPF", "Nome",

"Data-Nasc"])

dataframe2.show()
dataframe2.printSchema()

+------+-----+----------+
|   CPF| Nome| Data-Nasc|
+------+-----+----------+
|563214|Maria|2000-08-09|
|236541| João|1997-12-10|
|547812| José|1990-02-21|
+------+-----+----------+

root
 |-- CPF: long (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Data-Nasc: date (nullable = true)



## Inserindo colunas do DataFrame

In [48]:
from pyspark.sql.functions import *

new_dataf = dataframe2.withColumn("No Sorteio"

, round(rand()*10))

new_dataf.show()

+------+-----+----------+----------+
|   CPF| Nome| Data-Nasc|No Sorteio|
+------+-----+----------+----------+
|563214|Maria|2000-08-09|       2.0|
|236541| João|1997-12-10|       2.0|
|547812| José|1990-02-21|       1.0|
+------+-----+----------+----------+



In [49]:
dados_seguranca = sc.read.csv(
    path = "analise_homicidios.csv",
    inferSchema = True,
    header = True,
    sep = ';')
type(dados_seguranca)
dados_seguranca.show() #Visualizando os Dados
dados_seguranca.printSchema() #Visualizando o Schema
dados_seguranca.select('DATA').printSchema() #Visualizando o Schema (por índice)

+--------+----------+-----+----------------+----+-----+------------------+
|      No|      DATA| HORA|       INCIDENTE|SEXO|IDADE|         MUNICÍPIO|
+--------+----------+-----+----------------+----+-----+------------------+
|41290653|01/01/2020|03:27|HOMICÍDIO DOLOSO|   M|   27|        VILA VELHA|
|41291841|01/01/2020|08:10|HOMICÍDIO DOLOSO|   M|   39|             SERRA|
|41292282|01/01/2020|09:38|HOMICÍDIO DOLOSO|   M|   44|     BOA ESPERANCA|
|41292728|01/01/2020|10:49|HOMICÍDIO DOLOSO|   M|   42|BOM JESUS DO NORTE|
|41300399|02/01/2020|11:48|HOMICÍDIO DOLOSO|   F|   61|        VILA VELHA|
|41300416|02/01/2020|11:51|HOMICÍDIO DOLOSO|   M| null|         GUARAPARI|
|41304091|02/01/2020|18:46|HOMICÍDIO DOLOSO|   M|   36|        VILA VELHA|
|41304611|02/01/2020|19:43|HOMICÍDIO DOLOSO|   M|   51|        ECOPORANGA|
|41309912|03/01/2020|14:05|HOMICÍDIO DOLOSO|   M|   27|        VILA VELHA|
|41309912|03/01/2020|14:05|HOMICÍDIO DOLOSO|   M|   24|        VILA VELHA|
|41309504|03/01/2020|13:1

# Dataframe: Mudar o tipo de dados por colunas



In [51]:
from pyspark.sql.types import *

dados_seguranca1 = dados_seguranca.withColumn("IDADE", dados_seguranca[
"IDADE"].cast(IntegerType()))

for col in dados_seguranca1.dtypes:
  print(col[0]+" - "+col[1])

No - string
DATA - string
HORA - string
INCIDENTE - string
SEXO - string
IDADE - int
MUNICÍPIO - string


## Dataframe: Métodos e Atributos dos Dataframes
1. Contar Linhas no DataFrame:

In [53]:
from pyspark.sql.functions import *

rows = dados_seguranca.count()

print(f"Rows in Dataframe: {rows}")

Rows in Dataframe: 1164


## Dataframe: Métodos e Atributos dos Dataframes
2. Contar Linhas POR COLUNAS no DataFrame:

In [54]:
from pyspark.sql.functions import *

dados_seguranca.select(count(dados_seguranca.IDADE)).show()

+------------+
|count(IDADE)|
+------------+
|        1079|
+------------+



## Dataframe: Métodos e Atributos dos Dataframes
2. Contar Linhas POR COLUNAS no DataFrame:

In [55]:
from pyspark.sql.functions import *

dados_seguranca.select(count(dados_seguranca.SEXO)).show()

+-----------+
|count(SEXO)|
+-----------+
|       1162|
+-----------+



## Dataframe: Métodos e Atributos dos Dataframes
3. Médias (Average) por Colunas:

In [56]:
from pyspark.sql.functions import avg

dados_seguranca1.select(avg('IDADE')).collect()

[Row(avg(IDADE)=30.26893203883495)]